In [353]:
import sympy as sp

State space variables

In [354]:
iLm = sp.symbols("i_Lm")
vC = sp.symbols("v_C")

Input variables

In [355]:
iOut = sp.symbols("i_Out")
vIn = sp.symbols("v_In")

System parameters

In [356]:
Lm = sp.symbols("L_m")
Cf = sp.symbols("C_f")
Req = sp.symbols("R_eq")
Rl = sp.symbols("R_l")
n = sp.symbols("n")

In [357]:
A1 = sp.Matrix([[-Req / Lm, 0], [0, 0]])
A1

Matrix([
[-R_eq/L_m, 0],
[        0, 0]])

In [358]:
A2 = sp.Matrix([[0, -1 / (n * Lm)], [1 / (n * Cf), 0]])
A2

Matrix([
[        0, -1/(L_m*n)],
[1/(C_f*n),          0]])

In [359]:
B1 = sp.Matrix([[0, 1 / Lm], [-1 / Cf, 0]])
B1

Matrix([
[     0, 1/L_m],
[-1/C_f,     0]])

In [360]:
B2 = sp.Matrix([[0, 0], [-1 / Cf, 0]])
B2

Matrix([
[     0, 0],
[-1/C_f, 0]])

In [361]:
C1 = C2 = sp.Matrix([[0, 1]])
C1

Matrix([[0, 1]])

In [362]:
D1 = D2 = sp.Matrix([[0, 0]])
D1

Matrix([[0, 0]])

In [363]:
D0 = sp.symbols("D_0")

In [364]:
A = sp.simplify(A1 * D0 + A2 * (1 - D0))
A

Matrix([
[    -D_0*R_eq/L_m, (D_0 - 1)/(L_m*n)],
[(1 - D_0)/(C_f*n),                 0]])

In [365]:
B = sp.simplify(B1 * D0 + B2 * (1 - D0))
B

Matrix([
[     0, D_0/L_m],
[-1/C_f,       0]])

In [366]:
C = sp.simplify(C1 * D0 + C2 * (1 - D0))
C

Matrix([[0, 1]])

In [367]:
D = sp.simplify(D1 * D0 + D2 * (1 - D0))
D

Matrix([[0, 0]])

In [368]:
I_out = sp.symbols("I_out")
V_in = sp.symbols("V_in")

In [369]:
u0 = sp.Matrix([[I_out], [V_in]])
u0

Matrix([
[I_out],
[ V_in]])

In [370]:
x0 = -sp.simplify(A.inv() * B * u0)
x0

Matrix([
[                                          -I_out*n/(D_0 - 1)],
[-D_0*n*(D_0*V_in + I_out*R_eq*n - V_in)/(D_0**2 - 2*D_0 + 1)]])

In [371]:
E = sp.simplify((A1 - A2) * x0 + (B1 - B2) * u0)
E

Matrix([
[(-D_0*V_in - I_out*R_eq*n + V_in)/(L_m*(D_0**2 - 2*D_0 + 1))],
[                                       I_out/(C_f*(D_0 - 1))]])

In [372]:
F = sp.simplify((C1 - C2) * x0 + (D1 - D2) * u0)
F

Matrix([[0]])

In [373]:
new_B = sp.Matrix([[B, E]])
new_B

Matrix([
[     0, D_0/L_m, (-D_0*V_in - I_out*R_eq*n + V_in)/(L_m*(D_0**2 - 2*D_0 + 1))],
[-1/C_f,       0,                                        I_out/(C_f*(D_0 - 1))]])

In [374]:
new_D = sp.Matrix([[D, F]])
new_D

Matrix([[0, 0, 0]])

In [375]:
Rp = sp.symbols("R_p")
Rs = sp.symbols("R_s")
new_Req = D0 * Rp + Rs * (1 - D0) / n**2

In [376]:
s = sp.symbols("s")
I = sp.eye(2)
G = sp.simplify(C * (s * I - A).adjugate() * new_B + new_D)
G

Matrix([[(-D_0*R_eq - L_m*s)/(C_f*L_m), D_0*(1 - D_0)/(C_f*L_m*n), (D_0*I_out*R_eq*n + D_0*V_in + I_out*L_m*n*s + I_out*R_eq*n - V_in)/(C_f*L_m*n*(D_0 - 1))]])

In [377]:
num, den = sp.fraction((s * I - A).det())
f = sp.Poly(num / den, s)
f

Poly(s**2 + D_0*R_eq/L_m*s + (D_0**2 - 2*D_0 + 1)/(C_f*L_m*n**2), s, domain='ZZ(n,C_f,D_0,L_m,R_eq)')

In [378]:
f_s = sp.symbols("f_s")
deltaX = sp.simplify(
    ((A1 * D0 - A2 * (1 - D0)) * x0 + (B1 * D0 - B2 * (1 - D0)) * u0) / (4 * f_s)
)
deltaX

Matrix([
[D_0*(D_0*V_in + I_out*R_eq*n - V_in)/(2*L_m*f_s*(D_0 - 1))],
[                                    -D_0*I_out/(2*C_f*f_s)]])

In [379]:
deltaILm = sp.factor(deltaX[0])
deltaILm

D_0*(D_0*V_in + I_out*R_eq*n - V_in)/(2*L_m*f_s*(D_0 - 1))

In [380]:
PlossT = sp.symbols("P_loss_T")
ilmcalc = sp.sqrt(PlossT / Req)
ilmcalc

sqrt(P_loss_T/R_eq)

In [381]:
sp.Expr(sp.factor(sp.solve(deltaILm - ilmcalc, Lm)))

Expr([D_0*(D_0*V_in + I_out*R_eq*n - V_in)/(2*f_s*sqrt(P_loss_T/R_eq)*(D_0 - 1))])

In [382]:
deltaCf = sp.factor(deltaX[1])
deltaCf

-D_0*I_out/(2*C_f*f_s)

In [383]:
num, den = sp.fraction(sp.simplify(x0[1].subs({"R_eq": new_Req})))
Vout = sp.symbols("V_out")
sp.Poly(den * Vout - num, D0)

Poly((I_out*R_p*n**2 - I_out*R_s + V_in*n + V_out)*D_0**2 + (I_out*R_s - V_in*n - 2*V_out)*D_0 + V_out, D_0, domain='ZZ[n,I_out,R_p,R_s,V_in,V_out]')

In [422]:
kp = sp.symbols("k_p")
ki = sp.symbols("k_i")
kd = sp.symbols("k_d")
N = sp.symbols("N")

In [385]:
G = sp.simplify(C * (s * I - A).inv() * new_B + new_D)
G

Matrix([[n**2*(-D_0*R_eq - L_m*s)/(C_f*D_0*R_eq*n**2*s + C_f*L_m*n**2*s**2 + D_0**2 - 2*D_0 + 1), D_0*n*(1 - D_0)/(C_f*D_0*R_eq*n**2*s + C_f*L_m*n**2*s**2 + D_0**2 - 2*D_0 + 1), n*(I_out*n*(D_0*R_eq + L_m*s)*(D_0**2 - 2*D_0 + 1) + (D_0 - 1)**2*(D_0*V_in + I_out*R_eq*n - V_in))/((D_0 - 1)*(D_0**2 - 2*D_0 + 1)*(C_f*D_0*R_eq*n**2*s + C_f*L_m*n**2*s**2 + D_0**2 - 2*D_0 + 1))]])

In [425]:
# a1 = kp + N * kd
# a2 = ki + N * kp
# a3 = ki * N
a1 = sp.symbols("a_1")
a2 = sp.symbols("a_2")
a3 = sp.symbols("a_3")
a4 = sp.symbols("a_4")

In [489]:
PID = (a1 * s**2 + a2 * s + a3) / (s**2 + a4 * s)
PID
# PID = (a1 * s + a2) / s
# PID

(a_1*s**2 + a_2*s + a_3)/(a_4*s + s**2)

In [490]:
b1 = sp.symbols("b_1")
b2 = sp.symbols("b_2")
b3 = sp.symbols("b_3")
b4 = sp.symbols("b_4")

In [491]:
GDV = (b1 * s + b2) / (s**2 + b3 * s + b4)
GDV

(b_1*s + b_2)/(b_3*s + b_4 + s**2)

In [492]:
PID * GDV

(b_1*s + b_2)*(a_1*s**2 + a_2*s + a_3)/((a_4*s + s**2)*(b_3*s + b_4 + s**2))

In [493]:
Gsys = sp.simplify(PID * GDV / (1 + PID * GDV))
Gsys

(b_1*s + b_2)*(a_1*s**2 + a_2*s + a_3)/(s*(a_4 + s)*(b_3*s + b_4 + s**2) + (b_1*s + b_2)*(a_1*s**2 + a_2*s + a_3))

In [494]:
num, den = sp.fraction(Gsys)
num, den

((b_1*s + b_2)*(a_1*s**2 + a_2*s + a_3),
 s*(a_4 + s)*(b_3*s + b_4 + s**2) + (b_1*s + b_2)*(a_1*s**2 + a_2*s + a_3))

In [496]:
K = sp.symbols("K")
p = sp.symbols("p")

fw = (s + p) * (s**2 + a2 / a1 * s + a3 / a1) * (s + b2 / b1)
sp.Poly(den - fw, s)

Poly((a_1**2*b_1**2 + a_1*a_4*b_1 + a_1*b_1*b_3 - a_1*b_1*p - a_1*b_2 - a_2*b_1)/(a_1*b_1)*s**3 + (a_1**2*b_1*b_2 + a_1*a_2*b_1**2 + a_1*a_4*b_1*b_3 + a_1*b_1*b_4 - a_1*b_2*p - a_2*b_1*p - a_2*b_2 - a_3*b_1)/(a_1*b_1)*s**2 + (a_1*a_2*b_1*b_2 + a_1*a_3*b_1**2 + a_1*a_4*b_1*b_4 - a_2*b_2*p - a_3*b_1*p - a_3*b_2)/(a_1*b_1)*s + (a_1*a_3*b_1*b_2 - a_3*b_2*p)/(a_1*b_1), s, domain='ZZ(p,a_1,a_2,a_3,a_4,b_1,b_2,b_3,b_4)')